<a href="https://colab.research.google.com/github/derektorquette/Processamento-de-Lingua-Natural-UFMG/blob/main/7_Marca%C3%A7%C3%A3o_de_Classe_de_Palavra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mac-Morpho**

baixando o corpus a fonte é o NILC

In [ ]:
!wget http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
!tar zxvf macmorpho-v3.tgz

## serão baixados três arquivos, um conjunto de treinamento, um de desenvolvimento
## e outro arquivo de teste.

##carregando os conjuntos de treino, dev e teste.

In [ ]:
with open ('macmorpho-train.txt') as f: ##comando para abrir o arquivo que baixamos
    doc = f.read().split('\n') ## quebra de linha

doc[0].split()


## mesmo código, mas faremos uma tupla com a palavra e sua classe

In [ ]:
with open ('macmorpho-train.txt') as f: ##comando para abrir o arquivo que baixamos
    doc = f.read().split('\n') ## quebra de linha

[tuple(par.split('_')) for par in doc[0].split()]

In [ ]:
with open ('macmorpho-train.txt') as f: ##comando para abrir o arquivo que baixamos
    doc = f.read().split('\n') ## quebra de linha

traindata = []
for linha in doc:
  sentenca = [tuple(par.split('_')) for par in linha.split()]
  traindata.append(sentenca)

traindata[0] ## posso procurar outras setenças mundando o valor de 0

In [ ]:
with open ('macmorpho-train.txt') as f: #comando para abrir o conjunto treinamento
    doc = f.read().split('\n') ## quebra de linha

traindata = []
for linha in doc:
  sentenca = [tuple(par.split('_')) for par in linha.split()]
  traindata.append(sentenca)

with open ('macmorpho-dev.txt') as f: ##comando para abrir o conjunto dev
    doc = f.read().split('\n') ## quebra de linha

devdata = []
for linha in doc:
  sentenca = [tuple(par.split('_')) for par in linha.split()]
  devdata.append(sentenca)

with open ('macmorpho-test.txt') as f: ##comando para abrir o conjunto test
    doc = f.read().split('\n') ## quebra de linha

testdata = []
for linha in doc:
  sentenca = [tuple(par.split('_')) for par in linha.split()]
  testdata.append(sentenca)

corpus = traindata + devdata + testdata


## acessar uma sentença anotada no Mac-Morpho

In [ ]:
corpus[100]

In [ ]:
len(corpus) #tamanho do corpus

49934

In [ ]:
from collections import Counter ## contagem dos marcadores de palavras

tagset = []
for snt in corpus:
  for palavra in snt:
    tagset.append(palavra[1])

Counter(tagset)



## **Corpus UD_Portuguese-GSD**

baixando o corpus



In [ ]:
!git clone https://github.com/UniversalDependencies/UD_Portuguese-GSD.git

## carregando os conjuntos de treino, desenvolvimento e teste do corpus

In [ ]:
import os

def parse(fname):
  with open(fname) as f:
    doc = f.read()
  doc =  doc.split('\n\n')

  snts = []
  for j, inst in enumerate(doc[:-1]):
    snt = []
    rows = inst.split('\n')
    for i, elem in enumerate(rows):
        if elem[0] != '#':
          r = elem.split('\t')
          palavra, tag = r[1], r[3]
          if tag != '_':
            snt.append((palavra, tag))
    snts.append(snt)
  return snts

path = 'UD_Portuguese-GSD/'

traindata = parse(os.path.join(path, 'pt_gsd-ud-train.conllu'))
devdata = parse(os.path.join(path, 'pt_gsd-ud-dev.conllu'))
testdata = parse(os.path.join(path, 'pt_gsd-ud-test.conllu'))
corpus = traindata + devdata + testdata #juntando num corpus só


In [ ]:
len(corpus)

12020

In [ ]:
corpus[0]

In [ ]:
from collections import Counter ## contagem dos marcadores de palavras

tagset = []
for snt in corpus:
  for palavra in snt:
    tagset.append(palavra[1])

Counter(tagset)


##**Marcação de palavras baseada em regras nas UD**

In [ ]:
palavra2tag = {}

treino = traindata + devdata
for snt in treino:
  for par in snt:
    palavra = par[0].lower()
    classe = par[1]
    if palavra not in palavra2tag:
      palavra2tag[palavra] = []
    palavra2tag[palavra].append(classe)

for palavra in palavra2tag:
  tag_count = Counter(palavra2tag[palavra])
  most_frequent = sorted(tag_count.items(), key=lambda x: x[1], reverse=True) [0][0]
  palavra2tag[palavra] = most_frequent




In [ ]:
palavra2tag['eu']

'PRON'

In [ ]:
try:
  classe = palavra2tag['Thiago']
except:
  classe = 'NOUN'

In [ ]:
snt = 'pistoleiro de aluguel cobrava 500 dólar, pra mandar alguém pro beleléu, e com ele não havia xerife que parasse em pé, o xerife morria ou tinha que dar no pé'.split()
tags = []

for token in snt:
  try:
    classe = palavra2tag[token.lower()]
  except:
    classe = 'NOUN'
  tags.append(classe)

[(w, tags[i]) for i, w in enumerate(snt)] #apresenta alguns defeitos

## **Marcação de Palavras pelo Modelo Oculto de Márkov nas UD**

In [ ]:
import nltk

from nltk.tag import hmm

tagger = hmm.HiddenMarkovModelTrainer()
tagger = tagger.train(traindata + devdata)

In [ ]:
tagger.tag("hoje eu acordei feliz".split())

[('hoje', 'ADV'), ('eu', 'PRON'), ('acordei', 'DET'), ('feliz', 'DET')]

##**Spacy - para métodos de Deep Learning - Redes neurais**

o método já é treinado


In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy[cuda102]
!python3 -m spacy download pt_core_news_lg

In [ ]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")

In [ ]:
doc = nlp("ano passado eu morri, mas esse ano eu não morro.")

In [ ]:
for token in doc:
  print(token, token.pos_)

##**Inicializando o corpus para Avaliação**

In [ ]:
import os

def parse(fname):
  with open(fname) as f:
    doc = f.read()
  doc =  doc.split('\n\n')

  snts = []
  for j, inst in enumerate(doc[:-1]):
    snt = []
    rows = inst.split('\n')
    for i, elem in enumerate(rows):
        if elem[0] != '#':
          r = elem.split('\t')
          palavra, tag = r[1], r[3]
          if tag != '_':
            snt.append((palavra, tag))
    snts.append(snt)
  return snts

path = 'UD_Portuguese-GSD/'

traindata = parse(os.path.join(path, 'pt_gsd-ud-train.conllu'))
devdata = parse(os.path.join(path, 'pt_gsd-ud-dev.conllu'))
testdata = parse(os.path.join(path, 'pt_gsd-ud-test.conllu'))
corpus = traindata + devdata + testdata  #juntando num corpus só

**Método de avaliação Estado da Arte - Redes Neurais Profundas**

In [ ]:
import spacy
from spacy.tokens import Doc

spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")
nlp.tokenizer = lambda x: Doc(nlp.vocab, words=x.split())

y_real, y_pred = [], []
for sentence in testdata:
  for (token, pos) in sentence:
    y_real.append(pos)

  texto = ' '.join([w[0] for w in sentence])

  doc = nlp(texto)
  for token in doc:
    y_pred.append(token.pos_)

accscore = accuracy_score(y_real, y_pred)
print('Acurácia Redes Neurais: ', round(accscore, 2))